### prueba scrapear!

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent

#### carrefour

In [2]:
ua = UserAgent()
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2226.0 Safari/537.36'}

list_pages = list(range(0,336,24))
table_pages=[]
for number in list_pages:
    url = f'https://www.carrefour.es/supermercado/bebidas/cerveza/todas-las-cervezas/N-1uq8b5u/c?No={number}'
    html = requests.get(url, headers=header).text
    soup = BeautifulSoup(html, 'lxml')
    table = soup.find_all({'article':'product-card-item'})
    table_pages.append(table)


In [3]:
table_pages[0][24]

<article class="product-card-item">
<div class="load-more-products">
<p>SIGUIENTE PÁGINA</p>
<a class="pathCart" href="/supermercado/bebidas/cerveza/todas-las-cervezas/N-1uq8b5u/c?No=24&amp;Nr%3DAND%28product.shopCodes%3A004320%2Cproduct.salepointWithActivePrice_004320%3A1%2COR%28product.siteId%3AbasicSite%29%29OR%29">
</a>
</div>
</article>

In [4]:
len(table_pages[0])

25

In [39]:
data_raw=[]
for i in range(len(table_pages)):
    for j in range(len(table_pages[i])):
        rows = table_pages[i][j].find_all(['span','p','a'], {'class': ['price','price-less','format-price','js-gap-product-click-super']})
        promotion = table_pages[i][j].find_all(['p'], {'class': ['promocion-copy']})
        if len(rows) != 0:
            if len(promotion) != 0:
                data_raw.append(rows + promotion)
            else:
                data_raw.append(rows)
                
            

In [40]:
data_raw[20]

[<a class="js-gap-product-click-super" data-product-discount="" data-productid="526715416" data-special-campaign="qvpromo730001_feliz-aniversario|promof2320115_2a-al-50" href="/supermercado/cerveza-san-miguel-especial-lager-pack-de-12-latas-de-33-cl-san-miguel/R-526715416/p" title="Cerveza San Miguel especial Lager pack de 12 latas de 33 cl.">
 <img alt="Cerveza San Miguel especial Lager pack de 12 latas de 33 cl." aria-describedby="product-526715416" sizes="(max-width: 280px) 80px,(max-width: 490px) 100px, (max-width: 768px) 560px, (max-width: 991px) 280px, (max-width: 1280px) 560px" src="https://static.carrefour.es/hd_280x_/img_pim_food/198978_00_1.jpg" srcset="https://static.carrefour.es/hd_560x_/img_pim_food/198978_00_1.jpg 560w, https://static.carrefour.es/hd_280x_/img_pim_food/198978_00_1.jpg 280w, https://static.carrefour.es/hd_100x_/img_pim_food/198978_00_1.jpg 100w, https://static.carrefour.es/hd_100x_/img_pim_food/198978_00_1.jpg  80w" width="100%"/>
 </a>,
 <span class="pric

In [21]:
len(data_raw)

333

In [22]:
#price in euros
# price_raw = data_raw[0][1].text

def get_price(data_text):
    price_raw = data_text.text
    price = re.sub('\xa0€','',price_raw)
    price = re.sub('\n','',price)
    return re.sub(",",'.',price)

In [23]:
#price per liter
# price_l_raw = data_raw[0][2].text

def get_price_l(data_text):
    price_l_raw = data_text.text
    price_l = re.findall('[0-9]+,[0-9]+', price_l_raw)
    price_liter = re.sub(',','.',price_l[0])
    return price_liter

In [24]:
#title
# title_raw = data_raw[0][3].text

def get_title(data_text):
    title_raw = data_text.text
    title = re.sub("\n",'',title_raw)
    return title

In [55]:
#promotion product

def get_promotion(data_text):
        promotion_raw = data_text.text
        promotion = re.sub("\n",'',promotion_raw)
        return promotion


In [225]:
#brand
def get_brand(data_text):
    brand_raw = data_text.text
    brand = re.split('(Cerveza )|( pack)|( botella)|( lata)+|(cl)',brand_raw)
    return brand[6]

In [331]:
#container
def get_container(data_text):
    container_raw = data_text.text
    container = re.findall('botella|lata|barril',container_raw)
    if len(container)!=0:
        return container[0]
    else:
        return 'No specified'

In [363]:
#volumen
def get_volumen_unid(data_text):
    volumen_raw = data_text.text 
    volumen = re.findall('[0-9]+ cl',volumen_raw)
    if len(volumen)!=0:
        return volumen[0]
    else:
        volumen_l = re.findall('[0-9]+[ mcl]+',volumen_raw)
        return volumen_l[0]

In [390]:
#quantity
def get_quantity_pack(data_text):
    quantity_raw = data_text.text
    quantity_split = re.split('pack de',quantity_raw)
    if len(quantity_split)==2:
        quantity = re.findall('[0-9]+',quantity_split[1])
        return quantity[0]
    else:
        return 1

In [414]:
#image
def get_image(data_text):
    return data_text.find_all('img')[0].get('src')

In [424]:
data_beer_carrefour = pd.DataFrame(index=range(0,len(data_raw)),columns=['price','price_liter',
                                                                           'title','promotion','brand',
                                                                          'container','volumen_unid',
                                                                          'quantity_pack','image_url'])
for beer_number in range(len(data_raw)):
    data_beer_carrefour.iloc[beer_number,0] = get_price(data_raw[beer_number][1])
    data_beer_carrefour.iloc[beer_number,1] = get_price_l(data_raw[beer_number][2])
    data_beer_carrefour.iloc[beer_number,2] = get_title(data_raw[beer_number][3])
    data_beer_carrefour.iloc[beer_number,4] = get_brand(data_raw[beer_number][3])
    data_beer_carrefour.iloc[beer_number,5] = get_container(data_raw[beer_number][3])
    data_beer_carrefour.iloc[beer_number,6] = get_volumen_unid(data_raw[beer_number][3])
    data_beer_carrefour.iloc[beer_number,7] = get_quantity_pack(data_raw[beer_number][3])
    data_beer_carrefour.iloc[beer_number,8] = get_image(data_raw[beer_number][0])
    if len(data_raw[beer_number]) > 4:
        data_beer_carrefour.iloc[beer_number,3] = get_promotion(data_raw[beer_number][4])
    else:
        data_beer_carrefour.iloc[beer_number,3] = 'No promotion'


In [428]:
data_beer_carrefour.head()

,price,price_liter,title,promotion,brand,container,volumen_unid,quantity_pack,image_url
0,11.21,1.89,Cerveza Heineken Lager pack de 18 latas de 33 cl.,No promotion,Heineken Lager,lata,33 cl,18,https://static.carrefour.es/hd_280x_/img_pim_f...
1,14.80,1.60,Cerveza Cruzcampo Pilsen pack de 28 latas de 3...,No promotion,Cruzcampo Pilsen,lata,33 cl,28,https://static.carrefour.es/hd_280x_/img_pim_f...
2,5.15,1.56,Cerveza El Aguila pack de 10 latas de 33 cl.,No promotion,El Aguila,lata,33 cl,10,https://static.carrefour.es/hd_280x_/img_pim_f...
3,0.25,0.76,Cerveza Carrefour Holandesa lata 33 cl.,No promotion,Carrefour Holandesa,lata,33 cl,1,https://static.carrefour.es/hd_280x_/img_pim_f...
4,11.70,1.95,Cerveza Estrella Galicia especial pack de 24 b...,No promotion,Estrella Galicia especial,botella,25 cl,24,https://static.carrefour.es/hd_280x_/img_pim_f...
